<a href="https://colab.research.google.com/github/SamuelWanjiru/Recommender_System_Amazon_Reviews/blob/main/RecommenderSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<font color='red'> <u>Recommender System :</font> <font color='blue'>Amazon Reviews</u>**

E-commerce companies like AMazon , Walmart and eBay uses different recommendation systems to provide suggestions to the customers.Amazon uses currently item-item collaberrative filtering, which scales to massive datasets and produces high quality recommendation system in the real time. This system is a kind of a information filtering system which seeks to predict the "rating" or preferences which user is interested in.

**What are recommendation systems?**

* A recommender system, is a subclass of information filtering system that provide suggestions for items that are most pertinent to a particular user [info](https://en.wikipedia.org/wiki/Recommender_system#:~:text=A%20recommender%20system%2C%20or%20a,pertinent%20to%20a%20particular%20user.).
Recommeder system creates a similarity between the user and items and exploits the similarity between user/item to make recommendations.

* In this modern world we are overloaded with data and this data provides us the useful information. But it's not possible for the user to extract the information which interest them from these data. In order to help the user to find out information about the product , recommedation systems where developed.
